## Plan de bataille

* On fait un tableau csv avec un lien par ligne.
* On extrait tous les identifiants des facsimile.
* On les aligne en supprimant les lignes de IIIF inutiles.
* On resynchronise.

In [2]:
import csv
import json
import requests
import xml.etree.ElementTree as ET

## Récupérer le manifeste IIIF

Pour l'ark :
        https://gallica.bnf.fr/ark:/12148/bpt6k9684477n
le manifeste IIIF est :
        https://gallica.bnf.fr/iiif/ark:/12148/bpt6k9684477n/manifest.json
        
* Chemin vers les images :
        manifeste['sequences']['0']['canvases']
        
* Ensuite, "for each" depuis le précédent et ensuite, chemin :
        ['images']['0']['resource']['@id']
        
Et on a l'adresse de l'image en jpg.

In [17]:
requete = requests.get("https://gallica.bnf.fr/iiif/ark:/12148/bpt6k9684477n/manifest.json")
     
with open("/home/erminea/Documents/CONDE/iiif/pesnelle-t1.csv", "w") as csv_a_remplir:
    csv_contenu = csv.DictWriter(csv_a_remplir, fieldnames = ['label', 'iiif', 'transkribus'])
    csv_contenu.writeheader()

    for page in requete.json()['sequences'][0]['canvases']:
        
        csv_contenu.writerow({
            'label' : page['label'],
            'iiif': page['images'][0]['resource']['@id'],
            'transkribus':''
        })

## Récupérer les identifiants Transkribus

Viser les /TEI/facsimile/surface/graphic/@url

In [6]:
# On donne à ET l'espace de nom de la TEI utilisé dans nos XML.
correspondances = {}

pagedict = {}

with open('/home/erminea/Documents/CONDE/iiif/ruines.csv') as csvfile:
    csvlu = csv.DictReader(csvfile)
    
    for row in csvlu:
        pagination = row['label']
        iiif = row['iiif']
        ts = row['transkribus']
        
        localdict = {'iiif' : iiif, 'pagination' : pagination}
        pagedict[ts] = localdict
        del localdict


with open('/home/erminea/Documents/CONDE/editions/base-version/ruines_base.xml') as orig:
    # On lit l'arbre XML et on trouve sa racine ('TEI').
    tree = ET.parse(orig)
    root = tree.getroot()

    for surface in root.findall('.//surface'):
        image = surface.find('./graphic').get('url')
        
        surface.set('source', pagedict[image]['iiif'])
        surface.set('n', pagedict[image]['pagination'])
        
    tree.write('/home/erminea/Documents/CONDE/iiif/ruines_base.xml', xml_declaration=True, encoding="unicode")    